In [20]:
import sys

sys.path.append("../..")

from common.aidevs_auth import send_answer, get_token_and_task
from common.openai_auth import send_chat_completion, send_embeddings

In [25]:
token, task = get_token_and_task('knowledge')

In [26]:
task

{'code': 0,
 'msg': 'I will ask you a question about the exchange rate, the current population or general knowledge. Decide whether you will take your knowledge from external sources or from the knowledge of the model',
 'question': 'ile orientacyjnie ludzi mieszka w Polsce?',
 'database #1': 'Currency http://api.nbp.pl/en.html (use table A)',
 'database #2': "Knowledge about countries https://restcountries.com/ - field 'population'"}

In [27]:
resp = send_chat_completion(
    model_version="gpt-4", 
    system_content="Be ultra concise, return only one of categories: exchange rate|current population|general knowledge. Return JSON response." + task['msg'] + """###
What is the capital city of Poland?
{"category": "general knowledge"}
What is the population of USA?
{"category": "current population", "country": "Czech Republic"}
Please provide current GBPPLN rate.
{"category": "exchange rate", "currency": "GBP"}""",
    user_content=task['question']
)
resp

{'id': 'chatcmpl-8UOz32Fl2nj7iXQPI1hzlgZOpmx2n',
 'object': 'chat.completion',
 'created': 1702256569,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{"category": "current population", "country": "Poland"}'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 139, 'completion_tokens': 14, 'total_tokens': 153},
 'system_fingerprint': None}

In [28]:
import json, requests

classifications = json.loads(resp['choices'][0]['message']['content'])

res = None
if classifications['category'] == 'exchange rate':
    res = str(json.loads(requests.get("https://api.nbp.pl/api/exchangerates/rates/a/{}/?format=json".format(classifications['currency'])).text)["rates"][0]["mid"])
if classifications['category'] == 'current population':
    res = json.loads(requests.get(f"https://restcountries.com/v3.1/name/{classifications['country']}?fields=name,population").text)[0]['population']
if classifications['category'] == 'general knowledge':
    res = send_chat_completion(
        model_version="gpt-4", 
        system_content="Be ultra concise and provide answer briefly",
        user_content=task['question']
    )

In [29]:
res

37950802

In [30]:
send_answer(token, res)

'{\n    "code": 0,\n    "msg": "OK",\n    "note": "CORRECT"\n}'